In [3]:
import sys; sys.path

['/Users/abelsalau/scotiabankfrauddetection',
 '/Users/abelsalau/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles',
 '/Users/abelsalau/.vscode/extensions/ms-toolsai.jupyter-2022.11.1003412109/pythonFiles/lib/python',
 '/Users/abelsalau/opt/anaconda3/lib/python39.zip',
 '/Users/abelsalau/opt/anaconda3/lib/python3.9',
 '/Users/abelsalau/opt/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/Users/abelsalau/opt/anaconda3/lib/python3.9/site-packages',
 '/Users/abelsalau/opt/anaconda3/lib/python3.9/site-packages/aeosa',
 '/Users/abelsalau/opt/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg',
 '/Users/abelsalau/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions',
 '/Users/abelsalau/.ipython']

Imports that we are using

In [4]:
import os 
import warnings

import numpy as np
import tensorflow as tf
import pandas as pd
from pandas import Series, DataFrame

#visuals
import seaborn as sns
import matplotlib.pyplot as plt


Checking to see that tensorflow works. Your output should contain a tf.tensor

In [5]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(306.45374, shape=(), dtype=float32)


2023-01-20 13:17:54.957651: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Preliminary Graphing

In [6]:
#Loading the dataset - abel
url = "https://raw.githubusercontent.com/25ohms/scotiabankfrauddetection/main/Winter%202023%20Discovery%20Days/train_ScotiaDSD.csv"
data = pd.read_csv(url)

x = data[['EVENT_MONTH', 'EVENT_DAY_OF_WEEK','EVENT_TIME','TRANSACTION_ID', 
    'FRAUD_FLAG','AVAIL_CRDT', 'AMOUNT', 'CREDIT_LIMIT', 'CARD_NOT_PRESENT', 
      'USER_AGENT', 'CITY']]



credible_data = data[data['FRAUD_FLAG'] == 0.0] 
fraudulent_data = data[data['FRAUD_FLAG'] == 1.0]

In [7]:
print(credible_data.head())

   EVENT_MONTH  EVENT_DAY_OF_WEEK EVENT_TIME TRANSACTION_ID  FRAUD_FLAG  \
1            2                  4      12:51       8352728b         0.0   
2            2                  4      15:26       d50120e1         0.0   
3            2                  4      16:19       e96e2139         0.0   
4            2                  4      8:15:       32502a99         0.0   
5            2                  4      16:27       1b4856fb         0.0   

   AVAIL_CRDT  AMOUNT  CREDIT_LIMIT  CARD_NOT_PRESENT  \
1    20371.88   96.35       30700.0                 1   
2    15628.17  193.72       19500.0                 0   
3    12913.98   47.15       18400.0                 0   
4    26779.35  121.88       29200.0                 1   
5    27484.35    6.83       30700.0                 0   

                                          USER_AGENT  ...  \
1  Mozilla/5.0 (iPhone; CPU OS 16_0 like Mac OS X...  ...   
2                                                NaN  ...   
3                      

In [8]:
print(fraudulent_data.head())

     EVENT_MONTH  EVENT_DAY_OF_WEEK EVENT_TIME TRANSACTION_ID  FRAUD_FLAG  \
0              2                  4      14:16       a316accb         1.0   
24             2                  4      20:02       5fcc42f8         1.0   
36             2                  4      19:59       0d060041         1.0   
38             2                  4      12:30       74bbbe44         1.0   
132            2                  4      16:28       c264fb24         1.0   

     AVAIL_CRDT  AMOUNT  CREDIT_LIMIT  CARD_NOT_PRESENT  \
0        537.10   11.70       29200.0                 1   
24      9016.80   50.23       21400.0                 1   
36      3135.47   41.00       19500.0                 1   
38       674.70  161.95        1000.0                 1   
132     1314.30   44.08        5100.0                 1   

                                            USER_AGENT  ...  \
0                                                  NaN  ...   
24   Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...  

Checking to see if Data is null

In [9]:
total = data.isnull().sum().sort_values(ascending = False)
percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).transpose()

,USER_AGENT,CITY,EVENT_MONTH,MAX_PHONE_ORDER_PAST_30DAY,MEAN_TRAVEL_AND_ENTERTAINMENT_PAST_30DAY,MEAN_WEEKEND_PAST_30DAY,MAX_AUTO_PAST_30DAY,MAX_LS_PAST_30DAY,MAX_ELCTRNCS_PAST_30DAY,MAX_FASTFOOD_PAST_30DAY,...,MAX_RECREA_PAST_7DAY,MAX_HIGH_AMT_PAST_7DAY,MAX_HIGH_RECREA_PAST_7DAY,MAX_INTERNET_PAST_7DAY,MAX_PHONE_ORDER_PAST_7DAY,MAX_PURCHASE_EXCLUDING_GAS_PAST_7DAY,MAX_SWIPE_PAST_7DAY,MAX_WEEKEND_PAST_7DAY,STD_LX_PAST_7DAY,PREV_M_UNEMP_RATE
Total,77322.000000,77322.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,86.654713,86.654713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
num_fraudulent = fraudulent_data.shape[0]
num_total = data.shape[0]

p_fraudulent = num_fraudulent / num_total
print(p_fraudulent)

0.024139863274683402


Suggests that the data is heavily unbalanced.

In [12]:

mean_sum_max_rec = credible_data[['MEAN_RECREA_PAST_7DAY', 'MAX_RECREA_PAST_7DAY','SUM_RECREA_PAST_7DAY']]

print(mean_sum_max_rec)

       MEAN_RECREA_PAST_7DAY  MAX_RECREA_PAST_7DAY  SUM_RECREA_PAST_7DAY
1                       0.00                  0.00                  0.00
2                     113.26                208.65                215.47
3                      66.91                333.12                562.73
4                       0.00                  0.00                  0.00
5                      48.69                106.28                277.88
...                      ...                   ...                   ...
89225                  45.91                 56.55                137.74
89226                 123.51                225.50                247.03
89227                   0.00                  0.00                  0.00
89228                  57.88                 85.09                173.63
89229                  77.85                173.43                311.38

[87076 rows x 3 columns]
